In [2]:
import json

with open('key.json', 'r') as file:
    jsonKeys = json.load(file)

In [ ]:
import os
import base64

LANGFUSE_PUBLIC_KEY=jsonKeys['langfusePublic']
LANGFUSE_SECRET_KEY=jsonKeys['langfuseSecret']
LANGFUSE_AUTH=base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

os.environ["HF_TOKEN"] = jsonKeys['key']

# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
# # os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
# os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

In [5]:
from phoenix.otel import register
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
 
register(protocol="http/protobuf", 
         endpoint = "https://cloud.langfuse.com/api/public/otel/v1/traces", 
         headers = {"Authorization": f"Basic {LANGFUSE_AUTH}"}
         )
 
SmolagentsInstrumentor().instrument()

OpenTelemetry Tracing Details
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://cloud.langfuse.com/api/public/otel/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'authorization': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [7]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, FinalAnswerTool, HfApiModel, Tool, tool, VisitWebpageTool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.
    
    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }
    
    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)
    
    return best_service

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""
    
    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }
    
    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }
        
        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")




In [8]:
# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(), 
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool()
    ], 
    model=HfApiModel(),
    max_steps=10,
    verbosity_level=2
)


In [9]:
from smolagents import CodeAgent, HfApiModel

# agent = CodeAgent(tools=[], model=HfApiModel())
# alfred_agent = agent.from_hub('sergiopaniego/AlfredAgent', trust_remote_code=True)
agent.run("Give me the best playlist for a party at Wayne's mansion. The party idea is a 'villain masquerade' theme")  

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me the best playlist for a party at Wayne's mansion. The party idea is a 'villain masquerade' theme        │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I will use the `superhero_party_theme_generator` tool to get a unique theme idea for a 'villain           
masquerade' themed party. Then I will use the `web_search` tool to find the best playlist for that theme.          
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
theme_idea = superhero_party_theme_generator(category='villain masquerade')                                        
print(f"The suggested theme idea for the party is: {theme_idea}")                                                  
playlist_query = f"best party playlist for {theme_idea}"                                                           
playlist_result = web_search(query=playlist_query)                                                                 
print(f"The search result for the playlist is: {playlist_result}")                                                 
```<end_code>                                                                                                      
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  theme_idea = superhero_party_theme_generator(category='villain masquerade')                                      
  print(f"The suggested theme idea for the party is: {theme_idea}")                                                
  playlist_query = f"best party playlist for {theme_idea}"                                                         
  playlist_result = web_search(query=playlist_query)                                                               
  print(f"The search result for the playlist is: {playlist_result}")                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The suggested theme idea for the party is: Gotham Rogues' Ball: A mysterious masquerade where guests dress as 
classic Batman villains.
The search result for the playlist is: ## Search Results

[agents-course/units/en/unit2/smolagents/tools.mdx at main - 
GitHub](https://github.com/huggingface/agents-course/blob/main/units/en/unit2/smolagents/tools.mdx)
) print (result) # Output: "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman 
villains." With this tool, Alfred will be the ultimate super host, impressing his guests with a superhero-themed 
party they won't forget! 🦸‍♂️🦸‍♀️

[The 40 Greatest Batman Villains Of All Time - Screen 
Rant](https://screenrant.com/best-greatest-batman-villains-rogues-gallery-ever-all-time/)
The latest movie news on the movies you're most interested in seeing. Avatar's Zoe Saldaña reflects on playing 
Neytiri across five movies and for over two decades, calling it "a gift" to work …

[60 Best Party Themes for Adults (Creative and Unique)](https://www.thejoyofgifts.com/party-themes-for-adults/)
12. Masquerade Ball. ... Go gothic and elegant with a vampire-themed party. Guests can dress in dark, sophisticated
attire while enjoying blood-red cocktails and decadent, eerie treats like bleeding raspberry cheesecake bites. ... 
Transform your space into the dark, gritty streets of Gotham City for a Batman-themed party! Encourage guests to 
...

[Batman Rogues Gallery](https://batman.fandom.com/wiki/Batman_Rogues_Gallery)
The Rogues Gallery is the term for the various villains that Batman has faced over the years. These have ranged 
from costumed villains like the Joker, Poison Ivy and Two-Face, plainclothes crime bosses like Rupert Thorne, 
Carmine Falcone and Joe Chill to lesser known foes such as Magpie, Killer Moth and Humpty Dumpty.As enemies of the 
Batman are often sympathetic or otherwise morally complex ...

[11 Creative Masquerade Ball Party Ideas for an Enchanting 
Event](https://www.glastonburytheater.com/11-creative-masquerade-ball-party-ideas-for-an-enchanting-event/)
These culinary delights will elevate your masquerade ball party experience into one that truly captures the essence
of formal grandeur. [Blog Outlines]: 1. Introduction. 2. Venue Ideas for a Masquerade Ball Party. - Mysterious and 
unique venues - Plenty of dance space - Host a tea party - Rent masquerade costumes. 3.

[15 Creative Masquerade Party Ideas for Unforgettable Fun - This Makes 
That](https://thismakesthat.com/masquerade-party/)
You'll get creative masquerade party ideas that will make your event unforgettable. Masks, masks, everywhere. The 
key to a great masquerade party is, you guessed it, the masks! Hand out plain masks and some art supplies at the 
door for guests to decorate to their heart's content. You never know, you might be hosting the next Picasso! Next 
up, the venue. Think about glamming up your space with ...

[How to Host a Masquerade Theme Party - Party Themes 
Galore](https://partythemesgalore.com/how-to-host-a-masquerade-theme-party/)
Otherwise, get some music you and your guests love and have a fun playlist ready to roll! 4. Plan Masquerade Games 
or Activities Photo by Julio Rionaldo. Murder mysteries rule the masquerade theme party game scene. Here are a few 
fun digital downloads to check out: Murder at the Masquerade (10-20 characters) Masquerade Ball Murder Mystery 
(4-14 ...

[Gotham: 15 Major Villains, Ranked - Screen Rant](https://screenrant.com/gotham-every-major-villain-ranked/)
The latest movie news on the movies you're most interested in seeing. Avatar's Zoe Saldaña reflects on playing 
Neytiri across five movies and for over two decades, calling it "a gift" to work …

[Which of Batman's Rogues could throw Joker the best surprise party 
...](https://www.reddit.com/r/whowouldwin/comments/e5zsg9/which_of_batmans_rogues_could_throw_joker_the/)
Who could successfully set up the party without him knowing? If he only had one, which would be the b

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: Based on the search results, the most relevant playlist for a "villain masquerade" theme seems to be the  
"Batman villain arc" playlist on Spotify and the "Villain Mode" playlist. Both playlists contain a significant     
number of songs and have relatively high numbers of likes and saves.                                               
                                                                                                                   
I will choose the "Villain Mode" playlist as it has the largest number of saves and songs, which suggests it is    
highly popular and likely to have a good mix of villain-themed music.                                              
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
playlist_url = "https://open.spotify.com/playlist/37i9dQZF1DX3R7OWWGN4gH"                                          
final_answer(playlist_url)                                                                                         
```<end_code>                                                                                                      

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  playlist_url = "https://open.spotify.com/playlist/37i9dQZF1DX3R7OWWGN4gH"                                        
  final_answer(playlist_url)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: https://open.spotify.com/playlist/37i9dQZF1DX3R7OWWGN4gH

[Step 3: Duration 9.90 seconds| Input tokens: 14,938 | Output tokens: 489]

'https://open.spotify.com/playlist/37i9dQZF1DX3R7OWWGN4gH'

In [10]:
from langchain.agents import load_tools
from smolagents import CodeAgent, HfApiModel, Tool

search_tool = Tool.from_langchain(load_tools(["serpapi"])[0])

agent = CodeAgent(tools=[search_tool], model=HfApiModel())

agent.run("Search for luxury entertainment ideas for a superhero-themed event, such as live performances and interactive experiences.")

ModuleNotFoundError: Module langchain_community.agent_toolkits.load_tools not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`